In [2]:
import torch 
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
import pandas as pd
import numpy as np
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torch.utils.data import DataLoader


First, I will load my data and transform them to Tensors for PyTorch readability. I will define my transformations as follows.

In [ ]:
transformations = Compose([
    Resize((224, 224)), 
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize images
])